## Import required packages

In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dateutil
import matplotlib.pyplot as plt

## Load necessary datasets in form of pandas dataframe

In [17]:
## load residual measure data

# meeting minutes
df_measure_mm = pd.read_excel("../data/market_analysis_data/aggregate_measure_mm.xlsx", usecols=["ReleaseDate", "EndDate", "our_measure"])
df_measure_mm["EndDate"] = pd.to_datetime(df_measure_mm["EndDate"], format='%B/%d/%Y')
df_measure_mm["delay"] = (df_measure_mm["ReleaseDate"] - df_measure_mm["EndDate"]).dt.days

## load QQQ data
df_QQQ = pd.read_csv("../data/market_analysis_data/QQQ.csv")
df_QQQ["Date"] = pd.to_datetime(df_QQQ["Date"], format='%Y-%m-%d') 
# df_QQQ["Adj Close"]


In [27]:
df_merge = pd.merge(df_measure_mm, df_QQQ, left_on="ReleaseDate", right_on="Date", how="left")
df_merge["inter_meeting_period_ret"] = df_merge["Adj Close"].pct_change(-1)*-100 
df_merge = df_merge.dropna()
print(df_merge.shape)

var_x = df_merge["our_measure"].tolist()

dates_list = [df_merge["Date"].tolist()[0]]
QQQ_BAH_list = [100.0]
QQQ_LSM_list = [100.0]
for i in range(0, len(var_x)):
    if i+1 == len(var_x):
        dates_list.append(df_measure_mm["ReleaseDate"].tolist()[-1])
    else:
        dates_list.append(df_merge["ReleaseDate"].tolist()[i+1])
    measure_value = df_merge["our_measure"].tolist()[i]
    pct_change = df_merge["inter_meeting_period_ret"].tolist()[i]
    QQQ_BAH_list.append(QQQ_BAH_list[-1]*(1.0+(pct_change/100.0)))
    if measure_value <= 0.0: 
        QQQ_LSM_list.append(QQQ_LSM_list[-1]*(1.0+(pct_change/100.0)))
    else:
        QQQ_LSM_list.append(QQQ_LSM_list[-1]*(1.0-(pct_change/100.0)))

print(QQQ_BAH_list[-1], QQQ_LSM_list[-1])

fig = go.Figure()

fig.add_trace(go.Scatter(x=dates_list, y=QQQ_BAH_list,
    name="Buy and Hold"       # this sets its legend entry
))


fig.add_trace(go.Scatter(x=dates_list, y=QQQ_LSM_list,
    name="Our Strategy"
))



# Add figure title
fig.update_layout(
    title_text="Buy and Hold vs Our Strategy"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes title
fig.update_yaxes(title_text="Value of $100 Invested")

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    font=dict(
        size=25,
    ))

fig.show()



(189, 12)
84.87331363575859 195.46758053163788
